## Abstract

### Olist Marketing Funnel: Customer Segmentation, RFM Modeling, Persuasion Modeling, Targeted Recommendations with controlled A/B Testing, Social Media Analytics

A Data Driven Algorithmic Marketing system for brazil’s biggest online marketplace, to segment customers, to do Cohort Analysis, RFM Modeling, Uplift Modeling, Identify Upselling and Cross Selling opportunities using Targeted Recommendations, Context Aware Recommendation using product Features, Pricing Optimization. 


### Uplift or Pursuasion Modeling

Customer Segmentation helps Marketers to increase conversion and hence be cost-efficient. But imagine a case that you are about to launch a promotional campaign and you know which segment you want to target whether Big Spenders or Potential Loyalists. 
The question arises - Do you need to send the offer to everyone? No. 

In your current target group, there will be customers who are going to purchase anyways. You will cannibalize yourself by giving the promotion. We can summarize the segments based on this approach like below:

#### Treatment Responders: Customers that will purchase only if they receive an offer
#### Treatment Non-Responders: Customer that won’t purchase in any case
#### Control Responders: Customers that will purchase without an offer
#### Control Non-Responders: Customers that will not purchase if they don’t receive an offer


You need to target Treatment Responders (TR) and Control Non-Responders (CN). Since they won’t purchase unless you give an offer, these groups are boosting your uplift in promotional campaigns. On the other hand, you need to avoid targeting Treatment Non-Responders (TN) and Control Responders (CR). You will not benefit from targeting TN and, CN will make you cannibalize.



To identify which customers fall into which buckets, we use Uplift Modeling. 

In [1]:
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import re
from pandas.plotting import scatter_matrix
import plotly.offline as py
import plotly.graph_objs as go
import os

import sklearn
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime, timedelta,date
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
from sklearn.cluster import KMeans
%matplotlib inline


In [2]:
def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))


In [3]:
os.chdir(r'N:\Digital Marketing Analytics')

In [4]:
Olist_db = pd.read_csv('uplift_data.csv')
Olist_db

,customer_unique_id,recency,frequency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,0000b849f77a49e4a4ce2b2a4ca5be3f,14,5,432.90,0,0,Urban,0,Phone,No Offer,0
1,0000f46a3911fa3c0805444483337064,10,3,583.87,0,0,Urban,1,Web,No Offer,0
2,0004bd2a26a76fe21f786e4fbd80607f,1,4,480.54,0,0,Suburban,1,Web,Buy One Get One,0
3,00050ab1314c0e55a6ca13cf7181fecf,18,2,160.36,0,1,Rural,1,Web,Discount,0
4,0005ef4cd20d2893f0d9fbd94d3c0d97,40,1,187.91,0,1,Suburban,0,Web,Discount,0
...,...,...,...,...,...,...,...,...,...,...,...
41725,fffa431dd3fcdefea4b1777d114144f2,13,4,2992.10,0,0,Urban,1,Web,Buy One Get One,0
41726,fffb09418989a0dbff854a28163e47c6,24,1,329.38,1,1,Urban,1,Phone,Buy One Get One,0
41727,fffbf87b7a1a6fa8b03f081c5f51a201,4,2,199.50,0,0,Suburban,0,Phone,Discount,0
41728,fffcc512b7dfecaffd80f13614af1d16,30,1,45.10,1,1,Suburban,0,Phone,No Offer,0


In [5]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [6]:
Olist_db.conversion.mean()

0.14739995207284928

In [7]:
def calc_uplift(df):
    avg_order_value = 15
    
    #calculate conversions for each offer type
    base_conv = df[df.offer == 'No Offer']['conversion'].mean()
    disc_conv = df[df.offer == 'Discount']['conversion'].mean()
    bogo_conv = df[df.offer == 'Buy One Get One']['conversion'].mean()
    
    #calculate conversion uplift for discount and bogo
    disc_conv_uplift = disc_conv - base_conv
    bogo_conv_uplift = bogo_conv - base_conv
    
    #calculate order uplift
    disc_order_uplift = disc_conv_uplift * len(df[df.offer == 'Discount']['conversion'])
    bogo_order_uplift = bogo_conv_uplift * len(df[df.offer == 'Buy One Get One']['conversion'])
    
    #calculate revenue uplift
    disc_rev_uplift = disc_order_uplift * avg_order_value
    bogo_rev_uplift = bogo_order_uplift * avg_order_value
    
    
    print('Discount Conversion Uplift: {0}%'.format(np.round(disc_conv_uplift*100,2)))
    print('Discount Order Uplift: {0}'.format(np.round(disc_order_uplift,2)))
    print('Discount Revenue Uplift: ${0}\n'.format(np.round(disc_rev_uplift,2)))
    
    if len(df[df.offer == 'Buy One Get One']['conversion']) > 0:
          
        print('-------------- \n')
        print('BOGO Conversion Uplift: {0}%'.format(np.round(bogo_conv_uplift*100,2)))
        print('BOGO Order Uplift: {0}'.format(np.round(bogo_order_uplift,2)))
        print('BOGO Revenue Uplift: ${0}'.format(np.round(bogo_rev_uplift,2)))

Before building the model, let’s apply our calc_uplift function to see the current uplift of this campaign as a benchmark:

In [8]:
calc_uplift(Olist_db)

Discount Conversion Uplift: 0.21%
Discount Order Uplift: 29.28
Discount Revenue Uplift: $439.27

-------------- 

BOGO Conversion Uplift: -0.49%
BOGO Order Uplift: -69.62
BOGO Revenue Uplift: $-1044.25


Currently, our label is if a customer converted or not (1 or 0). We need to create four classes for TR, TN, CR, and CN. We know that the customers who received discount and bogo offers are Treatment and the rest is control. Let’s create a campaign_group column make this info visible:

In [9]:
Olist_db['campaign_group'] = 'treatment'
Olist_db.loc[Olist_db.offer == 'No Offer', 'campaign_group'] = 'control'

In [10]:
Olist_db['target_class'] = 0 #CN
Olist_db.loc[(Olist_db.campaign_group == 'control') & (Olist_db.conversion > 0),'target_class'] = 1 #CR
Olist_db.loc[(Olist_db.campaign_group == 'treatment') & (Olist_db.conversion == 0),'target_class'] = 2 #TN
Olist_db.loc[(Olist_db.campaign_group == 'treatment') & (Olist_db.conversion > 0),'target_class'] = 3 #TR

The mapping of the classes are below:
- 0 -> Control Non-Responders
- 1 -> Control Responders
- 2 -> Treatment Non-Responders
- 3 -> Treatment Responders

In [11]:
Olist_db.target_class.value_counts()

2    23875
0    11704
3     4112
1     2039
Name: target_class, dtype: int64

In [12]:
Olist_db.target_class.value_counts()/len(Olist_db)

2    0.572130
0    0.280470
3    0.098538
1    0.048862
Name: target_class, dtype: float64

In [13]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(Olist_db[['history']])
Olist_db['history_cluster'] = kmeans.predict(Olist_db[['history']])

In [14]:
Olist_db = order_cluster('history_cluster', 'history',Olist_db,True)
Olist_db.columns

Index(['customer_unique_id', 'recency', 'frequency', 'history',
       'used_discount', 'used_bogo', 'zip_code', 'is_referral', 'channel',
       'offer', 'conversion', 'campaign_group', 'target_class',
       'history_cluster'],
      dtype='object')

In [15]:
df_model = Olist_db.drop(['offer','campaign_group','conversion', 'customer_unique_id'],axis=1)
df_model = pd.get_dummies(df_model)

In [16]:
df_model.head()

,recency,frequency,history,used_discount,used_bogo,is_referral,target_class,history_cluster,zip_code_Rural,zip_code_Suburban,zip_code_Urban,channel_Phone,channel_Web
0,14,5,432.90,0,0,0,0,0,0,0,1,1,0
1,10,3,583.87,0,0,1,0,0,0,0,1,0,1
2,1,4,480.54,0,0,1,2,0,0,1,0,0,1
3,18,2,160.36,0,1,1,2,0,1,0,0,0,1
4,40,1,187.91,0,1,0,2,0,0,1,0,0,1


In [17]:
#create feature set and labels
X = df_model.drop(['target_class'],axis=1)
y = df_model.target_class

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
xgb_model = xgb.XGBClassifier().fit(X_train, y_train)
xgb_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [20]:
class_probs = xgb_model.predict_proba(X_test)
class_probs[0]

array([0.33609498, 0.00104272, 0.6618196 , 0.00104272], dtype=float32)

In [21]:
X_test['proba_CN'] = class_probs[:,0] 
X_test['proba_CR'] = class_probs[:,1] 
X_test['proba_TN'] = class_probs[:,2] 
X_test['proba_TR'] = class_probs[:,3]

In [22]:
X_test['uplift_score'] = X_test.eval('proba_CN + proba_TR - proba_TN - proba_CR')
X_test.head()

,recency,frequency,history,used_discount,used_bogo,is_referral,history_cluster,zip_code_Rural,zip_code_Suburban,zip_code_Urban,channel_Phone,channel_Web,proba_CN,proba_CR,proba_TN,proba_TR,uplift_score
23651,7,3,317.19,0,1,0,0,0,1,0,1,0,0.336095,0.001043,0.661820,0.001043,-0.325725
24383,9,2,150.58,0,1,1,0,0,1,0,0,1,0.333950,0.001046,0.663959,0.001046,-0.330009
4772,5,1,53.35,0,1,1,0,0,1,0,1,0,0.339212,0.001041,0.658705,0.001041,-0.319493
36183,1,4,863.10,0,0,0,1,0,0,1,1,0,0.342116,0.001040,0.655803,0.001040,-0.313688
16729,9,3,239.13,0,0,0,0,0,0,1,1,0,0.322277,0.001041,0.675641,0.001041,-0.353364


In [23]:
overall_proba = xgb_model.predict_proba(df_model.drop(['target_class'],axis=1))

In [24]:
df_model['proba_CN'] = overall_proba[:,0] 
df_model['proba_CR'] = overall_proba[:,1] 
df_model['proba_TN'] = overall_proba[:,2] 
df_model['proba_TR'] = overall_proba[:,3]

In [25]:
df_model['uplift_score'] = df_model.eval('proba_CN + proba_TR - proba_TN - proba_CR')

In [26]:
Olist_db['uplift_score'] = df_model['uplift_score']

In [27]:
Olist_db.head()

,customer_unique_id,recency,frequency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion,campaign_group,target_class,history_cluster,uplift_score
0,0000b849f77a49e4a4ce2b2a4ca5be3f,14,5,432.90,0,0,Urban,0,Phone,No Offer,0,control,0,0,-0.373584
1,0000f46a3911fa3c0805444483337064,10,3,583.87,0,0,Urban,1,Web,No Offer,0,control,0,0,-0.365396
2,0004bd2a26a76fe21f786e4fbd80607f,1,4,480.54,0,0,Suburban,1,Web,Buy One Get One,0,treatment,2,0,-0.350391
3,00050ab1314c0e55a6ca13cf7181fecf,18,2,160.36,0,1,Rural,1,Web,Discount,0,treatment,2,0,-0.339542
4,0005ef4cd20d2893f0d9fbd94d3c0d97,40,1,187.91,0,1,Suburban,0,Web,Discount,0,treatment,2,0,-0.299732


In [28]:
Olist_db.groupby('offer').uplift_score.mean()

offer
Buy One Get One   -0.245037
Discount          -0.241847
No Offer          -0.236025
Name: uplift_score, dtype: float32

### Model Evaluation
To evaluate our model, we will create two different groups and compare them with our benchmark. Groups are:
- 1- High Uplift Score: Customers have uplift score > 3rd quantile
- 2- Low Uplift Score: Customers have uplift score < 2nd quantile

We are going to compare:
1. Conversion uplift
2. Revenue uplift per target customer to see if our model can make our actions more efficient.

In [29]:
df_data_lift = Olist_db.copy()
uplift_q_75 = df_data_lift.uplift_score.quantile(0.75)
df_data_lift = df_data_lift[(df_data_lift.offer != 'Buy One Get One') & (df_data_lift.uplift_score > uplift_q_75)].reset_index(drop=True)

In [30]:
calc_uplift(df_data_lift)

Discount Conversion Uplift: 4.51%
Discount Order Uplift: 153.42
Discount Revenue Uplift: $2301.26



By using this model, we can easily make our campaign more efficient by:
1. Targeting specific segments based on the uplift score
2. Trying different offers based on customer’s uplift score